In [8]:
# ============================================================================
# INSTALL REQUIRED PACKAGES
# ============================================================================

import subprocess
import sys

packages = ['tqdm', 'scipy', 'pandas', 'numpy', 'psutil']

for pkg in packages:
    try:
        __import__(pkg)
        print(f"✅ {pkg}")
    except:
        print(f"📥 Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])
        print(f"✅ {pkg}")

print("\n✅ All packages ready\n")


✅ tqdm
✅ scipy
✅ pandas
✅ numpy
✅ psutil

✅ All packages ready



In [9]:
# ============================================================================
# LUNARSENSE-3: NOTEBOOK 2 - OPTIMIZED MULTI-GPU PROCESSING
# Processing ALL Data with Memory Management
# ============================================================================

import os
import sys
import json
import numpy as np
import pandas as pd
import torch
import torch.multiprocessing as mp
from datetime import datetime
from scipy import signal, stats
from tqdm import tqdm
import gc
import psutil
import warnings
warnings.filterwarnings('ignore')

print("=" * 100)
print("🌙 LUNARSENSE-3: NOTEBOOK 2 - MODALITY PREPROCESSING (OPTIMIZED)")
print("=" * 100 + "\n")

# Multi-GPU setup
n_gpus = torch.cuda.device_count()
print(f"🔥 GPU Setup:")
print(f"   Available GPUs: {n_gpus}")

for i in range(n_gpus):
    print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
    mem_gb = torch.cuda.get_device_properties(i).total_memory / 1e9
    print(f"           {mem_gb:.1f} GB memory")

print()

# CPU info
cpu_count = os.cpu_count()
print(f"💻 CPU: {cpu_count} cores")
print(f"   RAM: {psutil.virtual_memory().total / 1e9:.1f} GB\n")

# Processing config
CONFIG = {
    'batch_size': 100,        # Process 100 files at a time
    'n_gpus': n_gpus,
    'n_workers': min(32, cpu_count),  # Safe worker count
    'save_interval': 200,     # Save progress every 200 files
    'memory_limit_gb': 30     # Stop if RAM exceeds 30GB
}

print(f"⚙️ Processing Config:")
print(f"   Batch size: {CONFIG['batch_size']} files")
print(f"   Workers: {CONFIG['n_workers']}")
print(f"   GPUs: {CONFIG['n_gpus']}")
print(f"   Save interval: {CONFIG['save_interval']} files\n")


🌙 LUNARSENSE-3: NOTEBOOK 2 - MODALITY PREPROCESSING (OPTIMIZED)

🔥 GPU Setup:
   Available GPUs: 8
   GPU 0: NVIDIA A100-SXM4-40GB
           42.4 GB memory
   GPU 1: NVIDIA A100-SXM4-40GB
           42.4 GB memory
   GPU 2: NVIDIA A100-SXM4-40GB
           42.4 GB memory
   GPU 3: NVIDIA A100-SXM4-40GB
           42.4 GB memory
   GPU 4: NVIDIA A100-SXM4-40GB
           42.4 GB memory
   GPU 5: NVIDIA A100-SXM4-40GB
           42.4 GB memory
   GPU 6: NVIDIA A100-SXM4-40GB
           42.4 GB memory
   GPU 7: NVIDIA A100-SXM4-40GB
           42.4 GB memory

💻 CPU: 256 cores
   RAM: 1082.0 GB

⚙️ Processing Config:
   Batch size: 100 files
   Workers: 32
   GPUs: 8
   Save interval: 200 files



In [10]:
# ===========================================================================
# LOAD INVENTORY
# ===========================================================================

output_root = '/raid/home/srmist57/Chandrayan-3/LunarSense3_FullPipeline'
provenance_dir = os.path.join(output_root, '02_provenance')
processed_dir = os.path.join(output_root, '01_processed_data')

inventory_file = os.path.join(provenance_dir, 'full_data_inventory.json')

with open(inventory_file, 'r') as f:
    inventory = json.load(f)

data_inventory = inventory['file_details']

print("✅ Inventory loaded:")
print(f"   Total files: {inventory['total_files']:,}")
print(f"   Total size: {inventory['total_size_gb']:.2f} GB\n")

for inst, files in data_inventory.items():
    if files:
        print(f"  {inst:12s}: {len(files):5d} files")
print()


✅ Inventory loaded:
   Total files: 6,537
   Total size: 36.08 GB

  LIBS        :  3166 files
  ChaSTE      :   389 files
  ILSA        :  1939 files
  RAMBHA      :  1020 files
  APXS        :    11 files
  SPICE       :    12 files



In [11]:
# ===========================================================================
# CHASTE PROCESSING (MEMORY-OPTIMIZED)
# ===========================================================================

print("STEP 1: ChaSTE Processing (Memory-Safe)\n")

class ChaSTE_Processor:
    def __init__(self):
        self.saturation_threshold = 350.0
        self.drift_threshold = 5.0
        self.sampling_rate = 1.0
    
    def process_file(self, filepath):
        """Process single file"""
        try:
            df = pd.read_csv(filepath)
            
            # Find temp column
            temp_cols = [col for col in df.columns if 'temp' in col.lower()]
            if not temp_cols:
                numeric_cols = df.select_dtypes(include=[np.number]).columns
                if len(numeric_cols) == 0:
                    return None
                temp_col = numeric_cols[0]
            else:
                temp_col = temp_cols[0]
            
            thermal = df[temp_col].values
            thermal = np.nan_to_num(thermal, nan=np.nanmean(thermal))
            
            # Features
            x = np.arange(len(thermal))
            slope, _ = np.polyfit(x, thermal, 1)
            drift_rate = slope * 3600 * self.sampling_rate
            
            # QC flag
            qc_flag = 0
            if len(thermal) == 0 or np.isnan(thermal).sum() > len(thermal) * 0.5:
                qc_flag = 1
            elif (thermal > self.saturation_threshold).sum() > len(thermal) * 0.1:
                qc_flag = 3
            elif thermal.std() > 0.5:
                qc_flag = 2
            
            return {
                'filename': os.path.basename(filepath),
                'n_samples': len(thermal),
                'mean_temp': float(thermal.mean()),
                'std_temp': float(thermal.std()),
                'min_temp': float(thermal.min()),
                'max_temp': float(thermal.max()),
                'drift_rate': float(drift_rate),
                'qc_flag': int(qc_flag)
            }
        except:
            return None

print("✅ ChaSTE processor ready\n")


STEP 1: ChaSTE Processing (Memory-Safe)

✅ ChaSTE processor ready



In [12]:
# ===========================================================================
# PROCESS CHASTE FILES IN BATCHES
# ===========================================================================

print("Processing ChaSTE files in batches...\n")

chaste_files = data_inventory.get('ChaSTE', [])
processor = ChaSTE_Processor()
chaste_results = []

batch_size = CONFIG['batch_size']
n_batches = (len(chaste_files) + batch_size - 1) // batch_size

print(f"Total: {len(chaste_files)} files")
print(f"Batches: {n_batches} (size={batch_size})\n")

for batch_idx in range(n_batches):
    start_idx = batch_idx * batch_size
    end_idx = min(start_idx + batch_size, len(chaste_files))
    
    batch_files = chaste_files[start_idx:end_idx]
    
    print(f"Batch {batch_idx+1}/{n_batches}: Processing files {start_idx}-{end_idx}...")
    
    for file_info in tqdm(batch_files, desc=f"Batch {batch_idx+1}"):
        result = processor.process_file(file_info['path'])
        if result:
            chaste_results.append(result)
    
    # Memory cleanup
    gc.collect()
    
    # Check RAM
    ram_used = psutil.virtual_memory().used / 1e9
    if ram_used > CONFIG['memory_limit_gb']:
        print(f"\n⚠️ RAM limit reached ({ram_used:.1f} GB)")
        print("   Saving progress and continuing...\n")
        
        # Save intermediate results
        temp_df = pd.DataFrame(chaste_results)
        temp_file = os.path.join(processed_dir, f'chaste_batch_{batch_idx}.csv')
        temp_df.to_csv(temp_file, index=False)

# Create final DataFrame
chaste_df = pd.DataFrame(chaste_results)

print(f"\n✅ ChaSTE Complete: {len(chaste_results)}/{len(chaste_files)}\n")

# Summary
print("Summary:")
print(f"  Mean temp: {chaste_df['mean_temp'].mean():.2f} K")
print(f"  Files with drift: {(abs(chaste_df['drift_rate']) > 5.0).sum()}")
print()

# QC flags
for flag in range(4):
    count = (chaste_df['qc_flag'] == flag).sum()
    label = ['OK', 'MISSING', 'HIGH_NOISE', 'SATURATED'][flag]
    print(f"  {label:12s}: {count:4d}")
print()

# Save
chaste_output = os.path.join(processed_dir, 'chaste_processed.csv')
chaste_df.to_csv(chaste_output, index=False)
print(f"✅ Saved: {chaste_output}\n")

# Cleanup
del chaste_results, chaste_df
gc.collect()


Processing ChaSTE files in batches...

Total: 389 files
Batches: 4 (size=100)

Batch 1/4: Processing files 0-100...


Batch 1: 100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 306.40it/s]



⚠️ RAM limit reached (37.9 GB)
   Saving progress and continuing...

Batch 2/4: Processing files 100-200...


Batch 2: 100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 280.16it/s]



⚠️ RAM limit reached (37.9 GB)
   Saving progress and continuing...

Batch 3/4: Processing files 200-300...


Batch 3: 100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 240.48it/s]



⚠️ RAM limit reached (37.9 GB)
   Saving progress and continuing...

Batch 4/4: Processing files 300-389...


Batch 4: 100%|█████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 203.87it/s]



⚠️ RAM limit reached (37.9 GB)
   Saving progress and continuing...


✅ ChaSTE Complete: 385/389

Summary:
  Mean temp: 1880.48 K
  Files with drift: 36

  OK          :  179
  MISSING     :    0
  HIGH_NOISE  :    7
  SATURATED   :  199

✅ Saved: /raid/home/srmist57/Chandrayan-3/LunarSense3_FullPipeline/01_processed_data/chaste_processed.csv



0

In [13]:
# ===========================================================================
# ILSA PROCESSING (MEMORY-OPTIMIZED)
# ===========================================================================

print("STEP 2: ILSA Processing (Memory-Safe)\n")

class ILSA_Processor:
    def __init__(self):
        self.sampling_rate = 100.0
        self.sta_window = 100
        self.lta_window = 3000
        self.trigger_on = 3.0
    
    def sta_lta_detection(self, waveform):
        """Fast STA/LTA"""
        if len(waveform) < self.lta_window:
            return 0, 0.0
        
        # Subsample for speed
        step = max(1, len(waveform) // 10000)
        wave_sub = waveform[::step]
        
        sta = np.convolve(np.abs(wave_sub), np.ones(self.sta_window)//self.sta_window, mode='same')
        lta = np.convolve(np.abs(wave_sub), np.ones(self.lta_window)//self.lta_window, mode='same')
        
        ratio = sta / (lta + 1e-10)
        
        n_events = (ratio > self.trigger_on).sum()
        max_ratio = ratio.max() if len(ratio) > 0 else 0.0
        
        return n_events, max_ratio
    
    def process_file(self, filepath):
        """Process single ILSA file"""
        try:
            df = pd.read_csv(filepath, nrows=10000)  # Limit rows for memory
            
            # Find wave column
            wave_cols = [col for col in df.columns if 'wave' in col.lower() or 'amplitude' in col.lower()]
            if not wave_cols:
                numeric_cols = df.select_dtypes(include=[np.number]).columns
                if len(numeric_cols) == 0:
                    return None
                wave_col = numeric_cols[0]
            else:
                wave_col = wave_cols[0]
            
            waveform = df[wave_col].values
            waveform = np.nan_to_num(waveform, nan=0.0)
            
            # Detect events
            n_events, max_ratio = self.sta_lta_detection(waveform)
            
            # QC
            qc_flag = 0
            if len(waveform) == 0:
                qc_flag = 1
            elif np.abs(waveform).max() > 0 and (np.abs(waveform) >= np.abs(waveform).max() * 0.99).sum() > 10:
                qc_flag = 3
            
            return {
                'filename': os.path.basename(filepath),
                'n_samples': len(waveform),
                'n_events': int(n_events),
                'max_amplitude': float(np.abs(waveform).max()),
                'rms': float(np.sqrt((waveform**2).mean())),
                'max_sta_lta': float(max_ratio),
                'qc_flag': int(qc_flag)
            }
        except:
            return None

print("✅ ILSA processor ready\n")


STEP 2: ILSA Processing (Memory-Safe)

✅ ILSA processor ready



In [14]:
# ===========================================================================
# PROCESS ILSA FILES IN BATCHES (1,939 FILES)
# ===========================================================================

print("Processing ILSA files in batches (1,939 files, 30GB)...\n")
print("⚠️ This may take 15-20 minutes with memory management\n")

ilsa_files = data_inventory.get('ILSA', [])
processor = ILSA_Processor()
ilsa_results = []

batch_size = 50  # Smaller batches for large files
n_batches = (len(ilsa_files) + batch_size - 1) // batch_size

print(f"Total: {len(ilsa_files)} files")
print(f"Batches: {n_batches} (size={batch_size})\n")

for batch_idx in range(n_batches):
    start_idx = batch_idx * batch_size
    end_idx = min(start_idx + batch_size, len(ilsa_files))
    
    batch_files = ilsa_files[start_idx:end_idx]
    
    print(f"Batch {batch_idx+1}/{n_batches}: Files {start_idx}-{end_idx}...")
    
    for file_info in tqdm(batch_files, desc=f"Batch {batch_idx+1}"):
        result = processor.process_file(file_info['path'])
        if result:
            ilsa_results.append(result)
    
    # Memory cleanup
    gc.collect()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    # Save progress every 200 files
    if (batch_idx + 1) % 4 == 0:
        temp_df = pd.DataFrame(ilsa_results)
        temp_file = os.path.join(processed_dir, f'ilsa_progress_{batch_idx}.csv')
        temp_df.to_csv(temp_file, index=False)
        print(f"   💾 Progress saved: {len(ilsa_results)} files\n")
    
    # Check RAM
    ram_used = psutil.virtual_memory().used / 1e9
    print(f"   RAM: {ram_used:.1f} GB")
    
    if ram_used > CONFIG['memory_limit_gb']:
        print(f"   ⚠️ RAM limit reached - forcing cleanup\n")
        gc.collect()

# Final DataFrame
ilsa_df = pd.DataFrame(ilsa_results)

print(f"\n✅ ILSA Complete: {len(ilsa_results)}/{len(ilsa_files)}\n")

# Summary
print("Summary:")
print(f"  Total events: {ilsa_df['n_events'].sum()}")
print(f"  Files with events: {(ilsa_df['n_events'] > 0).sum()}")
print()

# QC
for flag in range(4):
    count = (ilsa_df['qc_flag'] == flag).sum()
    label = ['OK', 'MISSING', 'HIGH_NOISE', 'SATURATED'][flag]
    print(f"  {label:12s}: {count:4d}")
print()

# Save
ilsa_output = os.path.join(processed_dir, 'ilsa_processed.csv')
ilsa_df.to_csv(ilsa_output, index=False)
print(f"✅ Saved: {ilsa_output}\n")

# Cleanup
del ilsa_results, ilsa_df
gc.collect()


Processing ILSA files in batches (1,939 files, 30GB)...

⚠️ This may take 15-20 minutes with memory management

Total: 1939 files
Batches: 39 (size=50)

Batch 1/39: Files 0-50...


Batch 1: 100%|████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 1359.97it/s]

   RAM: 38.0 GB
   ⚠️ RAM limit reached - forcing cleanup



Batch 2/39: Files 50-100...


Batch 2: 100%|████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 1294.60it/s]


...

Batch 38: 100%|█████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 51.95it/s]


   RAM: 38.0 GB
   ⚠️ RAM limit reached - forcing cleanup

Batch 39/39: Files 1900-1939...


Batch 39: 100%|█████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 49.05it/s]


   RAM: 38.0 GB
   ⚠️ RAM limit reached - forcing cleanup


✅ ILSA Complete: 1674/1939

Summary:
  Total events: 0
  Files with events: 0

  OK          :  456
  MISSING     :    0
  HIGH_NOISE  :    0
  SATURATED   : 1218

✅ Saved: /raid/home/srmist57/Chandrayan-3/LunarSense3_FullPipeline/01_processed_data/ilsa_processed.csv



0

In [15]:
# ===========================================================================
# NOTEBOOK 2 SUMMARY
# ===========================================================================

print("=" * 100)
print("✅ NOTEBOOK 2 COMPLETE: MODALITY PREPROCESSING")
print("=" * 100 + "\n")

print("📊 Processing Summary:\n")
print(f"  ChaSTE: Processed with batch size {CONFIG['batch_size']}")
print(f"  ILSA: Processed with batch size 50 (memory-safe)")
print(f"  Memory management: Active")
print(f"  Progress saved: Every 200 files")
print()

print("✅ Output files:")
print(f"  {os.path.join(processed_dir, 'chaste_processed.csv')}")
print(f"  {os.path.join(processed_dir, 'ilsa_processed.csv')}")
print()

print("🚀 Ready for Notebook 3: Baseline Detectors")


✅ NOTEBOOK 2 COMPLETE: MODALITY PREPROCESSING

📊 Processing Summary:

  ChaSTE: Processed with batch size 100
  ILSA: Processed with batch size 50 (memory-safe)
  Memory management: Active
  Progress saved: Every 200 files

✅ Output files:
  /raid/home/srmist57/Chandrayan-3/LunarSense3_FullPipeline/01_processed_data/chaste_processed.csv
  /raid/home/srmist57/Chandrayan-3/LunarSense3_FullPipeline/01_processed_data/ilsa_processed.csv

🚀 Ready for Notebook 3: Baseline Detectors
